# Proyecto final: Clustering de usuarios de MyAnimeList: Hallando patrones y tendencias por comunidad

Nótese que, debido a la gran cantidad de información que debió ser extraída para realizar este
proyecto, se vio necesario crear variados archivos `jupyter notebook` adicionales para realizar la extracción
de datos. Todos ellos guardan la información necesaria en la carpeta `data/`.

Estos archivos deben ejecutarse en el siguiente orden:

Ruta primera (MyAnimeList):

1. `scraper_usuarios.ipynb`
2. `api_listas.ipynb`
3. `nombres_anime.ipynb`

Ruta segunda (AnimePlanet):

1. `nombres_anime_planet.ipynb`
2. `tags.ipynb`

Una vez terminadas ambas rutas al 100%, se puede proceder a ejecutar el archivo
`procesar_data.ipynb`. 

Es muy necesario para la efectiva realización de este proyecto. Dada la magnitud de este, y lo
exigente del tiempo para mandar los requests, se vio conveniente modularizar el proceso de
extracción de datos en varios archivos.

## Imports 

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from random import randint
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

## Contexto y motivación

En cualquier afición o actividad de interés con cierto nivel de popularidad, tienden a generarse
comunidades muy distintas que comparten el mismo pasatiempo en común. A fin de cuentas, los
humanos somos diferentes a lo largo del mundo, y existen hobbies de tan nivel de amplitud y variedad
que pueden coexistir múltiples razones para que sean atrayentes, pero... ¿cuáles son dichas razones?
¿Cuán heterogéneo resulta ser el universo interesado en cierto tema arbitrario? En este proyecto,
principalmente se tratará de identificar las diversas comunidades presentes en aquellos fans de la
animación japonesa.

La industria del anime moderno nació alrededor de la década de 1960, y desde entonces cada vez ha
gozado de una mayor popularidad. Según ciertas estadísticas (Ferjan, 2023), alrededor de más de un
tercio de la población mundial ve anime, lo que equivale a 2,88 mil millones de personas. Además, la
base de datos de artes multimedia japonesas cuenta con más de 136.700 obras bajo la categoría de
anime en sus registros. En los Estados Unidos, se estima que más del 50% de la generación Z ve anime
(Ferjan, 2023), y se proyecta que conforme avance el tiempo el reconocimiento de esta industria
ciertamente será mayor en nuestras vidas. En un ámbito más local, según un análisis de Google Trends,
se vio que Chile es el quinto país en el cual el anime es más popular (Ferjan, 2023).
Por otro lado, se procederá a dar un poco de contexto respecto a lo que es MyAnimeList. Es un sitio
web fundado en 2004 por el estadounidense Garrett Gyssler, que tiene como propósito primario el ser
una herramienta para que sus usuarios puedan registrar el anime y manga que han visto,
inscribiéndose en el concepto de **comunidad virtual de catalogación**; término que Urban Dictionary
define de la siguiente manera:

* *"Un subconjunto de los medios sociales, un sitio web o aplicación web que permite a los
usuarios catalogar digitalmente las cosas que les preocupan -música, libros, películas,
arte, gastronomía, artesanía, bricolaje, etc-, mientras que al mismo tiempo pueden
compartir su contenido en una comunidad en línea".* (Urban Dictionary, 2011)

Hoy en día, MyAnimeList es una compañía de bolsa, operada por el grupo Media Do desde 2019. La
creación de una cuenta de usuario con cerca de todas las funcionalidades asociadas es gratis, mas
también se ofrece un servicio mensual de 2,99 dólares para ser un “seguidor de MAL”; que ofrece los
beneficios de eliminación de anuncios de la página junto a la posibilidad de una mayor personalización
del perfil propio. Como grupo, creemos que estos beneficios que se obtienen por pagar mensualmente
son escasos y no valen mucho la pena, por lo que, frente a esta problemática, planteamos una factible
posible solución. Ella consiste en que se ofrezcan como funcionalidades pagadas el que sitio web le
permita al usuario saber al respecto sobre en cuál comunidad probablemente se identifique más; qué
serie, en base a lo anterior, le parezca mejor y sea recomendado que vea, entre un sinfín de otras
estadísticas personalizadas que se podrían obtener trabajando correctamente el enorme banco de
datos que sustenta MyAnimeList. Nótese que, muy recientemente, el sitio web ha implementado una
[página](https://myanimelist.net/profile/Xinil/statistics) de estadísticas personalizada para cada usuario, que justamente recoge la idea de que si se
desean saber todos los datos o gráficos sea necesario pagar. El inconveniente con ello es que aún
esas estadísticas que se ofrecen son muy simples a nuestro parecer; no siendo lo suficientemente
interesantes o con un trabajo serio por detrás; y es debido a ello que nos interesó la posibilidad de
ampliar masivamente la cantidad de información valiosa que podría estar ahí. Básicamente, el único
tipo de estadísticas que brinda MyAnimeList son del carácter descriptivo elemental, como cantidad de
animes visualizados (entre muchas otras cosas), promedios, distribuciones de calificaciones o
desviaciones estándar; medidas que cualquier persona podría fácilmente calcular o graficar con el
suficiente tiempo.

En síntesis, observamos una oportunidad actualmente malgastada en MyAnimeList, que podría
significar una importante ganancia económica para Media Do si es implementada correctamente, y
para ello es indispensable utilizar las herramientas y el conocimiento de Ciencia de Datos. A partir de
lo anterior, resulta evidente señalar a los trabajadores de Media Do como el público objetivo de este
proyecto, pues sin lugar a duda serían los mayores beneficiarios al ser ellos quienes lograrían mejorar
la rentabilidad de su compañía tomando como base las ideas desarrolladas por este trabajo. Como
comentario aparte, nótese que ello es coherente con las aspiraciones financieras de Media Do, dado
que un antecedente existente en 2019 (Ressler, 2019) da cuenta que dicha compañía tiene intenciones
de fortalecer MyAnimeList; mejorando las funcionalidades del sitio como una plataforma de marketing
y distribuidora de libros electrónicos. De manera indirecta, débase decirse que este proyecto tiene
como público a todas aquellas personas que les gustaría saber y estudiar las diversas comunidades y
sus características, que existen, como mínimo, entre los actuales usuarios de MyAnimeList (cerca de
15.106.951 cuentas bajo un registro del 9 de octubre de 2023 (MyAnimeList, 2023)).


No siendo una cuestión menos relevante, débase decirse que la inspiración inicial para escoger este
proyecto proviene de enterarse del proyecto hecho por el profesor Hernán Valdivieso, que
vigentemente realiza clases de Programación Avanzada y Visualización de Información en la Pontificia
Universidad Católica de Chile. Su proyecto consistió en, primeramente, armar un enorme dataset de
MyAnimeList, que recopiló en el siguiente [repositorio](https://github.com/Hernan4444/MyAnimeList-Database?tab=readme-ov-file#myanimelist-database-2020) de GitHub, titulado: “MyAnimeList-Database”.
Este repositorio cuenta con la información de la lista de animes de 325.772 usuarios diferentes, y
también con la información de 17.562 animes distintos. Nótese que dichos datos fueron extraídos en
parte (las preferencias de los usuarios) con la [Jikan API](https://jikan.docs.apiary.io/), entre
el 26 de febrero y el 20 de marzo de 2020. 

Sin embargo, para nuestro proyecto usaremos la [MyAnimeList API](https://myanimelist.net/apiconfig/references/api/v2), dado que es la oficial del
sitio web; siendo la Jikan una API no oficial. Ello tiene coherencia dado que la API oficial no estaba
disponible para su utilización en el momento donde el profesor hizo su dataset, puesto que en 2020
recién se habilitó desde el 6 de julio, según lo mencionado en el siguiente [post](https://myanimelist.net/forum/?topicid=1849732). A modo de abreviar,
considérese que escribir MAL es equivalente a escribir MyAnimeList.


Otro comentario importante que vale la pena destacar es la cuestión de la procedencia de las etiquetas.
Como contexto, cada entrada de anime en MAL cuenta con dos elementos cuya intención es resumir
lo fundamental o estilo de cada serie: géneros y temas. El problema con estos apartados es que
resultaría realmente difícil trabajar con ellos puesto que se requeriría realizar primero la labor previa
de comprobar la verdadera importancia de cada uno; además de que complejizaría innecesariamente
a posteriori un adecuado clustering. Para paliar dicha situación, el docente nos informó de que otro
sitio web, similar en funcionalidad a MAL, llamado [Anime-Planet](https://www.anime-planet.com/), cuenta con un sistema mejor de
etiquetación de sus animes. En este, existe un único apartado llamado -tags-, que podría decirse
que combina los géneros y temas que coloca MAL; y además suelen existir más (lo cual genera que se
pueda reconocer con una mayor facilidad las temáticas y “esencia” de la serie; característica
sumamente positiva para nuestra idea de proyecto). Se verá este aspecto inmediatamente a
continuación con un ejemplo (Welcome to the NHK!) del cual se tiene el conocimiento para aseverar
si es notoria la mejora de este apartado en Anime-Planet en comparación a MyAnimeList.


Géneros de Welcome to the NHK! en MAL: Comedy, Drama, Romance.


Temas de Welcome to the NHK! en MAL: Adult Cast, Otaku Culture, Psychological

Tags de Welcome to the NHK! en Anime-Planet: Comedy, Drama, Slice of Life, Dark Comedy, Loneliness,
Mental Illness, NEET, Otaku Culture, Parody, Psychological, Satire, Based on a Light Novel.

Se verifica exitosamente que son los tags de Anime-Planet los cuales captan de la forma más completa
y fiel lo tratado en la obra audiovisual.

A partir de dicho trabajo realizado, nos dimos cuenta de que podría ser muy interesante el
cumplimiento de ciertos objetivos; que en la posterior sección se explayarán.

Como documentación de los múltiples tags con los que cuenta Anime-Planet, en la siguiente [página](https://www.anime-planet.com/anime/tags?page=1)
se describe cada uno.


## Preguntas objetivo

De las preguntas originales que planteamos en la propuesta, las siguientes son las que hemos
conservado.

Nótese que se vio que las no consideradas resultarían muy complejas para lo realizado, o, no se tuvo
el tiempo suficiente para trabajar en ellas, dado un sinfín de dificultades que se encontraron de
manera inesperada realizando el trabajo final. Cuando decimos que dificultades, nos referimos a
infinitas e infinitas cuestiones que hicieron de todo nuestro trabajo uno extremadamentre complejo
y lento, en el cual se tuvieron que hacer sacrificios mentales del primer orden.

1. ¿Cuáles grupos de usuarios se pueden formar a partir de las reviews que ellos hacen respecto
a los animes vistos, los diferentes animes visualizados y sus tags asociados? ¿Siquiera existen
esos grupos? En caso afirmativo, ¿qué nombres específicos representarían la esencia de los
grupos hallados?

2. O incluso si existen, ¿son suficientemente significativas las diferencias como para hacer buenas
recomendaciones o predicciones de otras variables?

3. ¿Se pueden hallar patrones o tendencias regulares en las variables de los animes “clásicos”
(entendiéndose que estos cuentan con una excepcional recepción positiva o fama o ambos)?
¿Cuáles son dichos patrones? ¿Tienen sentido con lo que normalmente se esperaría?

4. ¿Está correlacionada la cantidad de animes vistos con la "comunidad" que se pertenece? Dicha
interrogante también es factible de plantear reemplazando la cantidad de animes vistos con
otras variables del proyecto.
   

## Datos

Para nuestros propósitos, usaremos originalmente tres distintos dataframes obtenidos de `data/`, y
a su vez estos generados con los archivos `jupyter notebook` mencionados inicialmente, que
sirvieron para su construcción efectiva.


Nótese que estos dataframes son de identidad muy similar a los descritos en la propuesta de
proyecto, así que no tienen casi ninguna novedad por sí mismos. De todos maneras, en en la sección
de análisis exploratorio se explicará con más detalle la información que contienen, y ciertos
aspectos preliminares que se pueden observar de ellos, en aspectos variados.   

Los dataframes:

1. `listas_usuarios.csv`: Contiene la información de las listas de anime de usuarios, indicando qué
   calificación le dio cada usuario a los animes que vio. Veáse con más detalle esto en la
   propuesta de proyecto. Este dataframe fue obtenido en base a la información existente en MyAnimeList.

2. `tags_animes.csv`: Contiene la información de los tags de cada anime, extraídos de AnimePlanet.
   En caso de que el anime presente un tag particular, este tendrá un 1 en la columna del tag; si
   no, tendrá un 0. Este dataframe fue obtenido en base a la información existente en AnimePlanet.
   
3. `detalles_animes.csv`: Contiene la información de los detalles de cada anime, extraídos de
   MyAnimeList. Contiene diversas columnas cuya explicación a detalle se encuentra en la propuesta
   de proyecto. Este dataframe fue obtenido en base a la información existente en MyAnimeList.


### Dataframe 1: `users_df.pickle`



In [9]:
with open(os.path.join("data", "users_df.pickle"), "rb") as archivo:
    users_df = pickle.load(archivo)

In [11]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8978 entries, 24 to 33183
Columns: 7793 entries, 39110:xxbladexx20 to 16656371:Aoto_uchiha10
dtypes: Sparse[float64, nan](7793)
memory usage: 6.5 MB


In [12]:
users_df

,39110:xxbladexx20,216949:haijuta,246587:d3athzero,107198:JayJay1401,45036:Seleare,4328:Joseph_,110895:Rudenick,93256:Moltke,160129:dander,256598:melandrea,...,16805797:iambabymango,16809902:Flopoflop,16791488:RhaenyraFR,16838607:SasOnator,17063891:Dezwhite05,17197026:dumb_zoro_,16617450:kotyboh,16459755:Tiberjuice,16965619:Watcher794_2,16656371:Aoto_uchiha10
24,10.0,NaN,10.0,NaN,NaN,NaN,NaN,NaN,8.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,8.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193,9.0,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
260,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
261,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Veáse que cada fila representa el id del anime en MyAnimeList, y cada columna representa el id del
usuario más su username. En cada celda, se encuentra la calificación que el usuario le dio al anime
en cuestión. Nótese que, si el usuario no ha calificado el anime, la celda tendrá un NaN, lo que es
natural considerando que existen muchos anime y es casi imposible que un usuario haya calificado
todos los que han sido recopilados.

### Dataframe 2: `tags_animes.csv`

In [ ]:
tags_animes = pd.read_csv(os.path.join('data', 'tags_animes.csv'))

### Dataframe 3: `detalles_animes.csv`

In [3]:
detalles_animes = pd.read_csv(os.path.join('data', 'MAL_dataframe.csv'))

In [13]:
detalles_animes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8978 entries, 0 to 8977
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  8978 non-null   int64  
 1   title               8978 non-null   object 
 2   main_picture        8978 non-null   object 
 3   alternative_titles  8978 non-null   object 
 4   start_date          8976 non-null   object 
 5   end_date            8977 non-null   object 
 6   synopsis            8900 non-null   object 
 7   mean                8908 non-null   float64
 8   rank                8474 non-null   float64
 9   popularity          8978 non-null   int64  
 10  num_list_users      8978 non-null   int64  
 11  num_scoring_users   8978 non-null   int64  
 12  media_type          8978 non-null   object 
 13  status              8978 non-null   object 
 14  num_episodes        8978 non-null   int64  
 15  start_season        8826 non-null   object 
 16  studio

Dicho dataframe recopila la id de cada anime, su nombre, y las características que consideramos
posiblemente relevantes para trabajar. 

## Análisis exploratorio de datos

### En Dataframe 1: `listas_usuarios.csv`

### En Dataframe 2: `tags_animes.csv`

### En Dataframe 3: `detalles_animes.csv`

## Análisis de datos:

### Introducción del clustering (recapitulación de la propuesta)

Para este proyecto, la herramienta estelar y basal que se usará es el clustering. Pero… ¿en qué consiste
el clustering? Buscando en internet se encontró una muy buena y clara definición escrita por Joaquín
Amat Rodrigo, un sénior Data Scientist español que cuenta con una web de divulgación de su área
bastante completa e informativa.

* *“El término clustering hace referencia a un amplio abanico de técnicas cuya finalidad es
encontrar patrones o grupos (clusters) dentro de un conjunto de observaciones. Las
particiones se establecen de forma que, las observaciones que están dentro de un mismo
grupo son similares entre ellas y distintas a las observaciones de otros grupos. Se trata
de un método de aprendizaje no supervisado (unsupervised), ya que el proceso no tiene
en cuenta a qué grupo pertenece realmente cada observación (si es que existe tal
información). Esta característica es la que diferencia al clustering de los métodos de
clasificación en el que sí emplea la verdadera clasificación durante su entrenamiento.”*
(Amat Rodrigo, 2020)

Como nuestra intención es precisamente encontrar conjuntos de usuarios similares de MyAnimeList,
es sumamente razonable que utilicemos dicha herramienta, la cual se encuentra implementada en la
librería scikit-learn de Python. En su página web, se describen los distintos algoritmos de clustering
que se pueden aplicar, mas se reconoce que en esta etapa del proyecto se desconoce cuál finalmente
se llegará a ocupar. Creemos que todavía nos falta el conocimiento necesario para responder dicha
interrogante y, además, solamente luego de realizar un análisis exhaustivo a los datos del trabajo se
podrá tomar una verdadera decisión más informada y consciente respecto a ello. Menciónese que
dicha librería también podrá sernos útil en el caso de que veamos conveniente realizar regresiones.
Primeramente, nuestro plan de proyecto parte con el paso de reunir los datos necesarios que
necesitamos para el cumplimiento de nuestros objetivos. Veamos que la fuente original de nuestros
datos se puede encontrar en sitios web, y, más específicamente, la data de MAL es factible de conseguir
fundamentalmente utilizando su API oficial, mientras que la data de Anime-Planet se tendrá que
conseguir bajo web scraping. Para estas labores, se utilizarán las librerías de pandas (para crear los
dataframes), requests y BeautifulSoup (obtención de información). Sin embargo, plantéese que un
inconveniente el cual tendremos que enfrentar es el hecho de que no es muy viable conseguir en un
reducido tiempo la data de la lista de usuarios de los más de 15 millones de usuarios de MyAnimeList.
Por lo tanto, se necesitará antes determinar una muestra aleatoria de los usuarios, y para ello una
posibilidad es usar la librería matemática de Numpy. Ya con todos los datos recopilados, puede pasarse
a la siguiente etapa.

Segundamente, lo elemental que debemos realizar para nuestro proyecto es una limpieza y
exploración de los datos que hayamos recolectado. Esto, es especialmente relevante teniendo en
cuenta el contexto de que los datos que trabajaremos serán obtenidos en base a la utilización de una
API y web scraping, y puede que en ambos procesos existan fallas inherentes en los valores extraídos
o una cantidad importante de información nula. Es nuestra responsabilidad revisar atenta y
minuciosamente cualquier tipo de desperfecto en dichos datos para corregirlos; así como igualmente
descartar posible información que lleguemos a considerar innecesaria y podría sobrecargar la carga
computacional asociada a los algoritmos que se usarán; disminuyendo y entorpeciendo nuestro
avance en el proyecto.


Terceramente, se piensa realizar lo ya mencionado, el susodicho clustering. De acuerdo con sus
resultados, se analizará cuál (si existe) es un nombre adecuado para cada grupo generado; viendo si
reúne las características habituales que siguen las comunidades de fans de la animación (gusto o
interés por una estética (temáticas) en concreto y/o juicio más o menos severo con respecto a lo
pensado de la calidad de las obras visualizadas).

----------------------------------------------------------------------------------------------------------------------------

Dicho texto, extraído de la propuesta de proyecto, efectivamente logró ser realidad. En específico,
la parte de descargar la información vía web scraping fue la más compleja de todas, y ciertamente
fue considerablemente lenta en general para la cantidad de información que se quería obtener. 

## Resumen de los resultados

## ¿Qué podría salir mal?

### Data insuficiente

Al momento de recolectar los usuarios de MAL, si bien se recolectó una amplia cantidad de usuarios,
puede que no haya sido lo suficiente para poder realizar un buen modelo de clustering. Ello puede
deberse a que se cree que igualmente existe una gran variedad en los gustos y estadísticas de los
usuarios de MAL, por lo que se necesitaría una cantidad de usuarios mucho mayor para el poder
identificar realmente todos los grupos o comunidades que existn en MAL. 

Como se dijo previamente en la sección de contexto, el sitio web cuenta con más de 15 millones de
cuentas, y, debido a lo lento y tedioso que supone el extraer la información de cada usuario,
solamente se consideró la información de cerca de 9000 usuarios. Si bien es una cantidad bastante
alta, en comparación a la cantidad total de usuarios, no es muy significativa. Por lo tanto, para
una mejor iteración del proyecto, se recomienda extraer más información de los usuarios, para así
llegar a mejores conclusiones.

Valdría la pena considerar posteriormente la diferencia en los resultados obtenidos al considerar
una mayor cantidad de usuarios, y si es que se obtienen resultados más precisos o no, y bueno,
realmente distintos o no. Nuestra suposición inicial es que dentro del sitio web podría existir una
muy amplia categorización de usuarios, lo que acentúa la necesidad de considerar una mayor muestra
para efectos prácticos. Aunque dicha suposición podría estar errónea; habría que comprobarlo.


En efecto, veáse lo siguiente:

In [5]:
print(f"El tamaño de la muestra elegida representa cerca de {9000/15000000*100}% de la base de datos de MyAnimeList.")

El tamaño de la muestra elegida representa cerca de 0.06% de la base de datos de MyAnimeList.


### Daño a la privacidad

En este proyecto, se utilizó información de usuarios de MAL, y se tiene en cuenta el username de
cada persona. En ese sentido, podría argumentarse que se está vulnerando en algún grado la privacidad de
los usuarios, puesto que se está utilizando información de ellos sin su consentimiento. Sin
embargo, se considera que no se está haciendo un mal uso de la información, puesto que dicha
información en principio no es sensible y se encuentra de público acceso en internet. 

Igualmente, en este proyecto para evitar cualquier problemática relacionada, podría haberse evitado
el trabajar con los nombres de usuario, y en su lugar haber utilizado un identificador único
aleatorio para cada usuario, que no tuviera relación con su nombre de usuario. Asimismo, otra forma
de solventar dicho problema habría sido consultar a la gente de MAL si están de acuerdos con el uso
que en este trabajo le dimos a su información, aunque ello habría tardado un tiempo considerable y
habría supuesto más dificultades logísticas por un lado.

Así, y considerando el libro de "Weapons of Math Destruction", es innegable que un uso masivo de la
información de usuarios podría acarrear consecuencias nocivas si no se tiene en consideración la
ética de la situación. Para este proyecto, las consecuencias no tienen un carácter tan grave, dado
que lo máximo que se podría hacer en principio con el clustering es colocar anuncios personalizados a los
usuarios de MAL, que consistirían en recomendarles animes que según el modelo les gusten en base a
su cercanía con cierta comunidad . Pero, si llegamos a considerar la fusión de esta información con otras fuentes con un nexo en común, ello
podría llegar a ser un problema si consideramos lo perjudicial de la publicidad invasiva y engañosa
que se puede generar. Se puede decir que es debido en parte a este último caso que suele ser
recomendable anonimizar los datos, para evitar que se pueda llegar a identificar a una persona en
cuestión en internet, dada la conexión que hoy en día se puede hacer con decenas de fuentes de
datos. 

### Búsqueda de patrones donde no existen en primer lugar

Como caso a considerar, puede llegar a existir el caso donde se busquen patrones en los datos, pero
la realidad es que estos estén muy separados desde un comienzo y no sean aptos para la aplicación
de modelos. Es decir, que no existan tendencias o comunidades que sirvan para el fin de nuestro
proyecto, y ciertamente nuestra pregunta no pueda tener una respuesta satisfactoria.

O, por otra parte, que las asociaciones sean tan intrincadas y difusas; complejas, que ningún
modelo existente al día de hoy las pueda identificar apropiadamente. Para resolver aquello, debería
aspirarse siempre a mejorar y variar los modelos utilizados para la Ciencia de Datos, y, no
quedarse todo el rato con lo mismo.

### Uso de datos que no desean ser usados

Dadas las dificultades inherentes que surgieron al realizar el scraping de los datos en
Anime-Planet, podría cuestionarse si verdaderamente el sitio web autoriza o no el uso de su
información para los fines aquí presentados. Por el momento, Anime-Planet no tiene una API oficial,
y la simulación requerida en los headers de las peticiones HTTP para poder realizar el scraping de
la información podría llegar a ser considerada como una violación a los términos de uso del sitio
web.

En ese sentido, se podría argumentar que el uso de los datos de Anime-Planet no es ético, y que
sería recomendable acercarse a los dueños del sitio web para consultarles si están de acuerdo con
lo realizado. Similarmente, sería una buena idea plantear a Anime-Planet la posibilidad de que abran una API
oficial, y se puedan evitar las problemáticas enfrentadas en este proyecto. 

### Problema de Escalabilidad

Este proyecto utiliza dataframes que, con la información de cerca de 9000 usuarios, ya tienen un tamaño
bastante considerable. Ahora bien, ¿qué pasaría si se quisiera escalar el proyecto a un nivel de
todos los usuarios de MyAnimeList en un momento dado? 

Creemos que ello requeriría de un excesivo tiempo de computación para el clustering, y no sería muy
cómodo de realizarse por un usuario común y corriente. La información que se utiliza sería excesiva
y probablemente cualuquier cambio hacia el modelo tardaría eternidades en hacerse efecto. Por
nuestros conocimientos actuales, si bien en el proyecto implementamos formas de paralelizar la
descarga de datos y reducir el tamaño de los dataframes (sparse matrices), no sabemos si existen
métodos aún mejores para manipular tales niveles de información.

Ello también convierte a este trabajo como uno imposible de cambiar seguidamente, a menos que
ciertos métodos de computación sean mejorados. Nos haría falta conocimientos avanzados de Big Data
para realizar un mejor trabajo. 